# Hyperparameter Tuning with Google Colab GPU

This notebook implements hyperparameter tuning for our ML project using Google Colab's GPU capabilities. Follow these steps to run the tuning process:

1. Mount Google Drive
2. Install dependencies
3. Import project files
4. Run hyperparameter tuning with GPU acceleration

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Import required libraries
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import os
import sys

# Add project directory to path
project_path = '/content/drive/MyDrive/path/to/your/project'  # Update this path
sys.path.append(project_path)

# Import project modules
import config
import cv_utils

In [ ]:
# GPU-accelerated logistic regression implementation
class LogisticRegressionGPU(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegressionGPU, self).__init__()
        self.linear = nn.Linear(input_dim, 1)
    
    def forward(self, x):
        return torch.sigmoid(self.linear(x))

def train_logistic_regression_gpu(X, y, lambda_, gamma, max_iters):
    # Convert data to PyTorch tensors and move to GPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    X_tensor = torch.FloatTensor(X).to(device)
    y_tensor = torch.FloatTensor(y).to(device)
    
    # Create dataset and dataloader
    dataset = TensorDataset(X_tensor, y_tensor)
    dataloader = DataLoader(dataset, batch_size=128, shuffle=True)
    
    # Initialize model
    model = LogisticRegressionGPU(X.shape[1]).to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=gamma)
    criterion = nn.BCELoss()
    
    # Training loop
    for epoch in range(max_iters):
        for batch_X, batch_y in dataloader:
            # Forward pass
            outputs = model(batch_X).squeeze()
            loss = criterion(outputs, batch_y)
            
            # L2 regularization
            l2_reg = torch.tensor(0.).to(device)
            for param in model.parameters():
                l2_reg += torch.norm(param)
            loss += lambda_ * l2_reg
            
            # Backward pass and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
    # Get weights and bias
    weights = model.linear.weight.data.cpu().numpy().flatten()
    bias = model.linear.bias.data.cpu().numpy()[0]
    
    return weights, bias

In [ ]:
# GPU-accelerated grid search with cross-validation
def grid_search_cv_gpu(X, y, lambda_grid=config.LAMBDA, gamma_grid=config.GAMMA, max_iters=config.MAX_ITERS):
    """
    Perform grid search over hyperparameters using cross-validation with GPU acceleration.
    """
    print(f"[Grid Search] Testing {len(lambda_grid) * len(gamma_grid)} hyperparameter combinations...")
    print(f"Using device: {'cuda' if torch.cuda.is_available() else 'cpu'}")
    
    results = []
    
    for lambda_ in lambda_grid:
        for gamma in gamma_grid:
            # Perform cross-validation for this parameter combination
            cv_results = cv_utils.cross_validate_logistic_regression(
                y, X, lambda_, gamma, max_iters,
                train_func=train_logistic_regression_gpu  # Use GPU-accelerated training
            )
            results.append(cv_results)
    
    # Find best configuration by F1 score
    best_result = max(results, key=lambda r: r['mean_f1'])
    
    print(f"\n[BEST CV] lambda={best_result['lambda']:.3e}, "
          f"gamma={best_result['learning_rate']:.3e}, "
          f"threshold={best_result['optimal_threshold']:.3f}")
    print(f"          F1={best_result['mean_f1']:.4f} (±{best_result['std_f1']:.4f}), "
          f"Acc={best_result['mean_accuracy']:.4f} (±{best_result['std_accuracy']:.4f})")
    
    return best_result, results

In [ ]:
# Load and preprocess data
def load_data():
    """Load training data from the data directory."""
    data_dir = os.path.join(project_path, 'data/dataset')
    
    # Load training data
    x_train = np.loadtxt(os.path.join(data_dir, 'x_train.csv'), delimiter=',', skiprows=1)
    y_train = np.loadtxt(os.path.join(data_dir, 'y_train.csv'), delimiter=',', skiprows=1)
    
    print(f"Loaded training data: X shape={x_train.shape}, y shape={y_train.shape}")
    return x_train, y_train

# Run hyperparameter tuning
def tune_gpu(X, y):
    """Main tuning function with GPU acceleration."""
    if config.HYPERPARAM_TUNING:
        best_result, results_list = grid_search_cv_gpu(X, y)
        save_tuning_results(best_result, results_list)
    else:
        best_result = load_tuning_results()
    
    return best_result

In [ ]:
# Run the hyperparameter tuning
if __name__ == "__main__":
    # Load data
    X, y = load_data()
    
    # Run tuning
    best_params = tune_gpu(X, y)
    
    print("\nBest parameters found:")
    print(f"Lambda: {best_params['lambda']:.6e}")
    print(f"Learning rate (gamma): {best_params['learning_rate']:.6e}")
    print(f"Optimal threshold: {best_params['optimal_threshold']:.6f}")
    print(f"F1 score: {best_params['mean_f1']:.4f} (±{best_params['std_f1']:.4f})")